### Training and Testing with Neuralhydrology
The following code uses the [`neuralhydrology`](https://neuralhydrology.readthedocs.io/en/latest/) python library to train and test LSTM streamflow models. An example of the code for the regional LSTM is provided.

In [ ]:
from neuralhydrology.nh_run import start_run, eval_run
import pickle
from pathlib import Path
import numpy as np
import os
import glob

In [ ]:
# hyperparameter grid search training for Experiment E with CA Data
ymls = os.listdir('Data/hyperparameters/e_ca_ymls/')
ymls_done = ['e_group_dam_sl_10_hs_121.yml',
             'e_group_dam_sl_10_hs_256.yml',
             'e_group_dam_sl_10_hs_32.yml',
             'e_group_dam_sl_365_hs_121.yml',
             'e_group_dam_sl_365_hs_256.yml',
             'e_group_dam_sl_365_hs_32.yml',
             'e_group_dam_sl_90_hs_121.yml']
nse = []
runs = []

for y in ymls:
    print(y)
    if y not in ymls_done:
        y_path = 'Data/hyperparameters/e_ca_ymls/' + y

        start_run(config_file=Path(y_path))

# eval_run(run_dir=Path('runs/e_group_dam_sl_90_hs_256_0802_134525'), period='test', epoch=50)
# eval_run(run_dir=Path('runs/e_group_flashy_sl_90_hs_256_0802_125052'), period='test', epoch=50)
# eval_run(run_dir=Path('runs/e_group_natural_sl_90_hs_32_0802_130259'), period='test', epoch=50)

In [ ]:
# hyperparameter grid search evaluating for Experiment A
nse = []
run_path = "runs/a_basins_2201_142415"
run_dir = Path(run_path)


for n in range(10, 26):
    print('epoch: ', n)
    # eval_run(run_dir=run_dir, period="validation", epoch=n)

    if n < 100:
        epoch_name = "model_epoch0" + str(n)
    
    else:
        epoch_name = "model_epoch" + str(n)

    with open(run_dir / "validation" / epoch_name / "validation_results.p", "rb") as fp:
        results = pickle.load(fp)

    nses = []
    for k in results.keys():
        if 'NSE' in results[k]['1D']:
            nses.append(results[k]['1D']['NSE'])

    # get the median NSE
    nse.append(np.nanmean(nses))

epoch_n = np.array(nse).argmax() + 10
print('Optimal epoch: ', epoch_n, 'NSE: ', np.max(nse))

eval_run(run_dir=run_dir, period="test", epoch=epoch_n)

### SHAP Feature Importance

This section provides the sample code to compute the SHAP values for a basin.

In [ ]:
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader
import random
import seaborn as sns

from neuralhydrology.evaluation.utils import load_scaler
from neuralhydrology.modelzoo.cudalstm import CudaLSTM
from neuralhydrology.utils.config import Config
from neuralhydrology.datasetzoo import get_dataset

In [ ]:
basin_id = "-14895675224"
seq_len = 90

config_file=Path("Data/hyperparameters/exp_e_ca_ymls/e_group_natural_sl_90_hs_121.yml")
cudalstm_config = Config(config_file)

# create a new model instance with random weights
cuda_lstm = CudaLSTM(cfg=cudalstm_config)

run_dir = Path("Data/runs/e_group_natural_sl_90_hs_121_2002_215342")

# load the trained weights into the new model.
model_path = run_dir / 'model_epoch050.pt'
model_weights = torch.load(str(model_path), map_location='cpu')
cuda_lstm.load_state_dict(model_weights)

# load the dataset
scaler = load_scaler(run_dir)
dataset = get_dataset(cudalstm_config, basin=basin_id, is_train=False, period='train', scaler=scaler)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True, collate_fn=dataset.collate_fn)

dataset_test = get_dataset(cudalstm_config, basin=basin_id, is_train=False, period='test', scaler=scaler)
dataloader_test = DataLoader(dataset_test, batch_size=1, shuffle=False, collate_fn=dataset.collate_fn)

################################################################################

sample = next(iter(dataloader))

################################################################################

M = 1000
counter = 0
n_features = sample['x_d'].shape[2]
n = sample['x_d'].shape[1]
features = list(range(n_features))

shapley_ts = {}
for i in range(0, n_features):
    for k in range(0, n):
        key = str(i) + '_' + str(k)
        shapley_ts[key] = []

for sample_test in dataloader_test:
    if counter > 731:
        break
    counter += 1
    if counter > 718:
        print('Sample: ', counter)
        # feature_dict = {}
        x = sample_test['x_d']
        for j in features:
            marginal_contributions = {}
            phi_j_x = []

            for i in range(0, n):
                marginal_contributions[i] = []
            
            feature_idxs = list(range(n_features))
            feature_idxs.remove(j)
            
            for _ in range(M):
                z = next(iter(dataloader))['x_d']
                x_idx = random.sample(feature_idxs, min(max(int(0.2*n_features), random.choice(feature_idxs)), int(0.8*n_features)))
                z_idx = [idx for idx in feature_idxs if idx not in x_idx]

                # construct two new instances
                x_plus_j = np.array([(x[:,:,i][0]).numpy() if i in x_idx + [j] else (z[:,:,i][0]).numpy() for i in range(n_features)]).reshape((seq_len, n_features))
                x_minus_j = np.array([(z[:,:,i][0]).numpy() if i in x_idx + [j] else (x[:,:,i][0]).numpy() for i in range(n_features)]).reshape((seq_len, n_features))
                x_plus_j = np.expand_dims(x_plus_j, 0)
                x_minus_j = np.expand_dims(x_minus_j, 0)
                
                # calculate marginal contribution
                s_minus_j = {'x_d': torch.Tensor(x_minus_j).cpu(), 'y': sample_test['y'], 'date': sample_test['date'],
                            'x_s': sample_test['x_s']}
                y_hat_minus_j = cuda_lstm(s_minus_j)['y_hat']
                
                s_plus_j = {'x_d': torch.Tensor(x_plus_j).cpu(), 'y': sample_test['y'], 'date': sample_test['date'],
                            'x_s': sample_test['x_s']}
                y_hat_plus_j = cuda_lstm(s_plus_j)['y_hat']
                
                marginal_contribution = y_hat_plus_j - y_hat_minus_j

                for k in range(0, n):
                    marginal_contributions[k].append(float(marginal_contribution[:,k,:][0][0].detach().cpu()))

            for l in range(0, n):
                phi_j_x.append(sum(marginal_contributions[l])/len(marginal_contributions[l]))

            print(f"Shaply value for feature {j}:", phi_j_x)

            for i in shapley_ts.keys():
                a, b = i.split('_')[0], i.split('_')[1]
                if a == str(j):
                    shapley_ts[i].append(phi_j_x[int(b)])

        shap_name = 'Data/shaps/shapley_ts_' + basin_id + '_' + str(counter) + '.csv'
        pd.DataFrame(dict([(k,pd.Series(v)) for k,v in shapley_ts.items() ])).to_csv(shap_name, index=False)